# 트럼프 트위터 데이터 수집

In [9]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import json, html
from datetime import datetime, timezone
import time
import pandas as pd
from selenium.common.exceptions import InvalidSessionIdException, WebDriverException

BASE_URL = (
    "https://www.thetrumparchive.com/?utm_source=chatgpt.com"
    "&retweet=%22false%22"
    "&device=%22All+devices%22"
    "&resultOffset=0"
    "&resultssortOption=%22Latest%22"
    "&startDate=%222016-11-08%22"
    "&endDate=%222017-01-20%22"
)


SCROLL_PAUSE_SEC = 2.0     # 스크롤 후 대기 시간
MAX_EMPTY_SCROLLS = 5      # 새로 나오는 트윗이 없을 때 몇 번까지 버틸지

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 15)

all_tweets = []
seen_ids = set()

try:
    print("페이지 열기:", BASE_URL)
    driver.get(BASE_URL)

    # 첫 로딩에서 트윗 요소가 뜰 때까지 대기
    try:
        wait.until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "div.ttaTweet")
            )
        )
    except Exception:
        print("처음 로딩에서 트윗 요소를 찾지 못했음. 종료.")
    
    empty_scrolls = 0

    while True:
        # 현재까지 페이지 전체 HTML 파싱
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, "html.parser")

        new_in_this_round = 0
        for div in soup.select("div.ttaTweet"):
            raw = div.get("data-tweet")
            if not raw:
                continue

            data = json.loads(html.unescape(raw))
            ts = data.get("date")
            tweet_id = data.get("id")

            if ts is None or tweet_id is None:
                continue

            # 중복 체크
            if tweet_id in seen_ids:
                continue
            seen_ids.add(tweet_id)

            dt_utc = datetime.fromtimestamp(ts / 1000.0, tz=timezone.utc)

            inner_html = html.unescape(data.get("text", ""))
            inner_soup = BeautifulSoup(inner_html, "html.parser")
            text = inner_soup.get_text(" ", strip=True)

            t = {
                "datetime_utc": dt_utc.strftime("%Y-%m-%d %H:%M:%S"),
                "id": tweet_id,
                "text": text,
                "retweets": data.get("retweets"),
                "favorites": data.get("favorites"),
                "source": data.get("source"),
            }
            all_tweets.append(t)
            new_in_this_round += 1

        print(f"이번 라운드에서 새로 수집한 트윗 수: {new_in_this_round}")
        print(f"현재까지 고유 트윗 수(id 기준): {len(seen_ids)}")

        if new_in_this_round == 0:
            empty_scrolls += 1
            print(f"새로 나온 트윗 없음. empty_scrolls={empty_scrolls}")
            if empty_scrolls >= MAX_EMPTY_SCROLLS:
                print("여러 번 스크롤해도 새 트윗이 안 나와서 종료.")
                break
        else:
            empty_scrolls = 0

        # 스크롤 맨 아래로 내려서 다음 트윗들 로딩 유도
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)

except InvalidSessionIdException:
    print("중간에 브라우저 세션이 끊어졌어요. 여기까지 저장합니다.")

finally:
    try:
        driver.quit()
    except:
        pass

    print("\n최종 수집된 트윗 개수(고유 id 기준):", len(seen_ids))
    df = pd.DataFrame(all_tweets)

    if not df.empty:
        df = df.drop_duplicates(subset=["id"]).reset_index(drop=True)
        print("중복 제거 후 최종 행 수:", len(df))

    df.to_csv(
        r"C:\Users\shinchaewon\Desktop\trump_tweets_2017_2021_full.csv",
        index=False,
        encoding="utf-8-sig",
    )
    print("CSV 저장 완료")


페이지 열기: https://www.thetrumparchive.com/?utm_source=chatgpt.com&retweet=%22false%22&device=%22All+devices%22&resultOffset=0&resultssortOption=%22Latest%22&startDate=%222016-11-08%22&endDate=%222017-01-20%22


C:\Users\shinchaewon\AppData\Local\Temp\ipykernel_15004\413371047.py:74: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  inner_soup = BeautifulSoup(inner_html, "html.parser")


이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 25
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 50
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 75
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 100
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 125
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 150
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 175
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 200
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 225
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 250
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 275
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 300
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 325
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 350
이번 라운드에서 새로 수집한 트윗 수: 14
현재까지 고유 트윗 수(id 기준): 364
이번 라운드에서 새로 수집한 트윗 수: 0
현재까지 고유 트윗 수(id 기준): 364
새로 나온 트윗 없음. empty_scrolls=1
이번 라운드에서 새로 수집한 트윗 수: 0
현재까지 고유 트윗 수(id 기준): 364
새로 나온 트윗 없음. empty_scrolls=2
이번 라운드에서 새로 수집한 트윗 수: 0
현재까지 고유 트윗 수(id 기준): 364
새로 나온 트윗 없음. empty_scrolls=3
이번 라운드에서 새로 수집한 트윗 

In [3]:
import pandas as pd

df = pd.read_csv(r"C:\Users\shinchaewon\Desktop\trump_tweets_2017_2021_full.csv")

print("행 수:", len(df))
print("고유 id 개수:", df['id'].nunique())

vc = df['id'].value_counts()
print("중복된 id 개수:", (vc > 1).sum())
print("중복 예시:")
print(vc[vc > 1].head())


행 수: 9325
고유 id 개수: 9325
중복된 id 개수: 0
중복 예시:
Series([], Name: count, dtype: int64)


In [14]:

BASE_URL = (
    "https://www.thetrumparchive.com/?utm_source=chatgpt.com"
    "&startDate=%222017-01-20%22"
    "&endDate=%222019-07-05%22"
    "&retweet=%22false%22"
    "&device=%22All+devices%22"
)

SCROLL_PAUSE_SEC = 2.0     # 스크롤 후 대기 시간
MAX_EMPTY_SCROLLS = 5      # 새로 나오는 트윗이 없을 때 몇 번까지 버틸지

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 15)

all_tweets = []
seen_ids = set()

try:
    print("페이지 열기:", BASE_URL)
    driver.get(BASE_URL)

    # 첫 로딩에서 트윗 요소가 뜰 때까지 대기
    try:
        wait.until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "div.ttaTweet")
            )
        )
    except Exception:
        print("처음 로딩에서 트윗 요소를 찾지 못했음. 종료.")
    
    empty_scrolls = 0

    while True:
        # 현재까지 페이지 전체 HTML 파싱
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, "html.parser")

        new_in_this_round = 0
        for div in soup.select("div.ttaTweet"):
            raw = div.get("data-tweet")
            if not raw:
                continue

            data = json.loads(html.unescape(raw))
            ts = data.get("date")
            tweet_id = data.get("id")

            if ts is None or tweet_id is None:
                continue

            # 중복 체크
            if tweet_id in seen_ids:
                continue
            seen_ids.add(tweet_id)

            dt_utc = datetime.fromtimestamp(ts / 1000.0, tz=timezone.utc)

            inner_html = html.unescape(data.get("text", ""))
            inner_soup = BeautifulSoup(inner_html, "html.parser")
            text = inner_soup.get_text(" ", strip=True)

            t = {
                "datetime_utc": dt_utc.strftime("%Y-%m-%d %H:%M:%S"),
                "id": tweet_id,
                "text": text,
                "retweets": data.get("retweets"),
                "favorites": data.get("favorites"),
                "source": data.get("source"),
            }
            all_tweets.append(t)
            new_in_this_round += 1

        print(f"이번 라운드에서 새로 수집한 트윗 수: {new_in_this_round}")
        print(f"현재까지 고유 트윗 수(id 기준): {len(seen_ids)}")

        if new_in_this_round == 0:
            empty_scrolls += 1
            print(f"새로 나온 트윗 없음. empty_scrolls={empty_scrolls}")
            if empty_scrolls >= MAX_EMPTY_SCROLLS:
                print("여러 번 스크롤해도 새 트윗이 안 나와서 종료.")
                break
        else:
            empty_scrolls = 0

        # 스크롤 맨 아래로 내려서 다음 트윗들 로딩 유도
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)

except InvalidSessionIdException:
    print("중간에 브라우저 세션이 끊어졌어요. 여기까지 저장합니다.")

finally:
    try:
        driver.quit()
    except:
        pass

    print("\n최종 수집된 트윗 개수(고유 id 기준):", len(seen_ids))
    df = pd.DataFrame(all_tweets)

    if not df.empty:
        df = df.drop_duplicates(subset=["id"]).reset_index(drop=True)
        print("중복 제거 후 최종 행 수:", len(df))

    df.to_csv(
        r"C:\Users\shinchaewon\Desktop\trump_tweets_2017_2019_full.csv",
        index=False,
        encoding="utf-8-sig",
    )
    print("CSV 저장 완료")

페이지 열기: https://www.thetrumparchive.com/?utm_source=chatgpt.com&startDate=%222017-01-20%22&endDate=%222019-07-05%22&retweet=%22false%22&device=%22All+devices%22


C:\Users\shinchaewon\AppData\Local\Temp\ipykernel_15444\1406428268.py:60: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  inner_soup = BeautifulSoup(inner_html, "html.parser")
C:\Users\shinchaewon\AppData\Local\Temp\ipykernel_15444\1406428268.py:60: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  inner_soup = BeautifulSoup(inner_html, "html.parser")


이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 25
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 50
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 75
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 100
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 125
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 150
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 175
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 200
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 225
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 250
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 275
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 300
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 325
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 350
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 375
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 400
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 425
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 450
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 475
이번 라운드에서 새로 수집한 트윗 수: 25
현재까지 고유 트윗 수(id 기준): 500
이번 

In [22]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import json, html
from datetime import datetime, timezone
import time
import pandas as pd
from selenium.common.exceptions import InvalidSessionIdException

BASE_URL = (
    "https://www.thetrumparchive.com/?utm_source=chatgpt.com"
    "&startDate=%222017-01-20%22"
    "&endDate=%222019-02-10%22"
    "&retweet=%22false%22"
    "&device=%22All+devices%22"
)

SCROLL_PAUSE_SEC = 1.5
MAX_EMPTY_SCROLLS = 10

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20)

all_tweets = []
seen_ids = set()

try:
    print("페이지 열기:", BASE_URL)
    driver.get(BASE_URL)

    # 트윗 하나라도 뜰 때까지 기다리기
    first_tweet = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.ttaTweet"))
    )

    # 첫 번째 트윗 기준으로, 실제로 스크롤되는 부모 컨테이너 찾기
    scroll_target = driver.execute_script("""
        let el = arguments[0];
        while (el) {
            const style = window.getComputedStyle(el);
            const overflowY = style.overflowY;
            if ((overflowY === 'auto' || overflowY === 'scroll') &&
                (el.scrollHeight - el.clientHeight) > 50) {
                return el;
            }
            el = el.parentElement;
        }
        return document.scrollingElement || document.body;
    """, first_tweet)

    print("스크롤 타겟 찾음")

    empty_scrolls = 0

    while True:
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, "html.parser")

        new_in_this_round = 0

        for div in soup.select("div.ttaTweet"):
            raw = div.get("data-tweet")
            if not raw:
                continue

            data = json.loads(html.unescape(raw))
            ts = data.get("date")
            tweet_id = data.get("id")

            if ts is None or tweet_id is None:
                continue

            if tweet_id in seen_ids:
                continue
            seen_ids.add(tweet_id)

            dt_utc = datetime.fromtimestamp(ts / 1000.0, tz=timezone.utc)

            inner_html = html.unescape(data.get("text", ""))
            inner_soup = BeautifulSoup(inner_html, "html.parser")
            text = inner_soup.get_text(" ", strip=True)

            t = {
                "datetime_utc": dt_utc.strftime("%Y-%m-%d %H:%M:%S"),
                "id": tweet_id,
                "text": text,
                "retweets": data.get("retweets"),
                "favorites": data.get("favorites"),
                "source": data.get("source"),
            }
            all_tweets.append(t)
            new_in_this_round += 1

        print(f"이번 라운드 새 트윗: {new_in_this_round} | 누적 고유 id: {len(seen_ids)}")

        if new_in_this_round == 0:
            empty_scrolls += 1
            print(f"새로 나온 트윗 없음. empty_scrolls={empty_scrolls}")
            if empty_scrolls >= MAX_EMPTY_SCROLLS:
                print("여러 번 스크롤해도 새 트윗이 안 나와서 종료.")
                break
        else:
            empty_scrolls = 0

        # 실제 스크롤 컨테이너를 아래로 내리기
        driver.execute_script(
            "arguments[0].scrollTop = arguments[0].scrollHeight;",
            scroll_target
        )
        time.sleep(SCROLL_PAUSE_SEC)

except InvalidSessionIdException:
    print("중간에 브라우저 세션이 끊어졌어요. 여기까지 저장합니다.")

finally:
    try:
        driver.quit()
    except:
        pass

    print("\n최종 수집된 트윗 개수(고유 id 기준):", len(seen_ids))
    df = pd.DataFrame(all_tweets)

    if not df.empty:
        df = df.drop_duplicates(subset=["id"]).reset_index(drop=True)
        print("중복 제거 후 최종 행 수:", len(df))

        df.to_csv(
            r"C:\Users\shinchaewon\Desktop\trump_tweets_2017_20190210_fixed.csv",
            index=False,
            encoding="utf-8-sig",
        )
        print("CSV 저장 완료")
    else:
        print("수집된 데이터가 없습니다.")


페이지 열기: https://www.thetrumparchive.com/?utm_source=chatgpt.com&startDate=%222017-01-20%22&endDate=%222019-02-10%22&retweet=%22false%22&device=%22All+devices%22
스크롤 타겟 찾음
이번 라운드 새 트윗: 25 | 누적 고유 id: 25


C:\Users\shinchaewon\AppData\Local\Temp\ipykernel_15444\3026932916.py:82: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  inner_soup = BeautifulSoup(inner_html, "html.parser")
C:\Users\shinchaewon\AppData\Local\Temp\ipykernel_15444\3026932916.py:82: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  inner_soup = BeautifulSoup(inner_html, "html.parser")


이번 라운드 새 트윗: 25 | 누적 고유 id: 50
이번 라운드 새 트윗: 25 | 누적 고유 id: 75
이번 라운드 새 트윗: 25 | 누적 고유 id: 100
이번 라운드 새 트윗: 25 | 누적 고유 id: 125
이번 라운드 새 트윗: 25 | 누적 고유 id: 150
이번 라운드 새 트윗: 25 | 누적 고유 id: 175
이번 라운드 새 트윗: 25 | 누적 고유 id: 200
이번 라운드 새 트윗: 25 | 누적 고유 id: 225
이번 라운드 새 트윗: 25 | 누적 고유 id: 250
이번 라운드 새 트윗: 25 | 누적 고유 id: 275
이번 라운드 새 트윗: 25 | 누적 고유 id: 300
이번 라운드 새 트윗: 25 | 누적 고유 id: 325
이번 라운드 새 트윗: 25 | 누적 고유 id: 350
이번 라운드 새 트윗: 25 | 누적 고유 id: 375
이번 라운드 새 트윗: 25 | 누적 고유 id: 400
이번 라운드 새 트윗: 25 | 누적 고유 id: 425
이번 라운드 새 트윗: 25 | 누적 고유 id: 450
이번 라운드 새 트윗: 25 | 누적 고유 id: 475
이번 라운드 새 트윗: 25 | 누적 고유 id: 500
이번 라운드 새 트윗: 25 | 누적 고유 id: 525
이번 라운드 새 트윗: 25 | 누적 고유 id: 550
이번 라운드 새 트윗: 25 | 누적 고유 id: 575
이번 라운드 새 트윗: 25 | 누적 고유 id: 600
이번 라운드 새 트윗: 25 | 누적 고유 id: 625
이번 라운드 새 트윗: 25 | 누적 고유 id: 650
이번 라운드 새 트윗: 25 | 누적 고유 id: 675
이번 라운드 새 트윗: 25 | 누적 고유 id: 700
이번 라운드 새 트윗: 25 | 누적 고유 id: 725
이번 라운드 새 트윗: 25 | 누적 고유 id: 750
이번 라운드 새 트윗: 25 | 누적 고유 id: 775
이번 라운드 새 트윗: 25 | 누적 고유 id: 800
이번 라운드 새 트

In [11]:
import pandas as pd

# 1) 파일 로드
df1 = pd.read_csv(r"C:\Users\shinchaewon\Desktop\텍스트마이닝\trump_tweets_2017_2021_merged_sorted.csv")
df2 = pd.read_csv(r"C:\Users\shinchaewon\Desktop\trump_tweets_2017_2021_full.csv")

# 2) 하나로 합치기
df = pd.concat([df1, df2], ignore_index=True)

print("합치기 전 전체 행수:", len(df))

# 3) 중복 제거 (id 기준)
df = df.drop_duplicates(subset=["id"]).reset_index(drop=True)

print("중복 제거 후 행수:", len(df))

# 4) datetime_utc → datetime 타입으로 변환
df["datetime_utc"] = pd.to_datetime(df["datetime_utc"], errors="coerce")

# 5) 시간 순으로 정렬
df = df.sort_values(by="datetime_utc").reset_index(drop=True)

# 6) 최종 저장
output_path = r"C:\Users\shinchaewon\Desktop\trump_tweets_2016_2021.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print("완성:", output_path)


합치기 전 전체 행수: 16924
중복 제거 후 행수: 16924
완성: C:\Users\shinchaewon\Desktop\trump_tweets_2016_2021.csv


In [15]:
import pandas as pd

# 1) 파일 로드
df1 = pd.read_csv(r"C:\Users\shinchaewon\Desktop\텍스트마이닝\trump_tweets_2017_2021_merged_sorted.csv")
df2 = pd.read_csv(r"C:\Users\shinchaewon\Desktop\trump_tweets_2017_2021_full.csv")

s = df['datetime_utc'].astype(str).str.strip()
s = s.str.replace(' UTC', '', regex=False)

df['datetime_utc'] = pd.to_datetime(s, errors='raise', utc=True)

print("df1 columns:", df1.columns.tolist())
print("df2 columns:", df2.columns.tolist())

# 2) 하나로 합치기
df = pd.concat([df1, df2], ignore_index=True)
print("합치기 전 전체 행수:", len(df))

# 3) 중복 제거 (id 기준)
df = df.drop_duplicates(subset=["id"]).reset_index(drop=True)
print("중복 제거 후 행수:", len(df))

# 4) datetime 변환
df["datetime_utc"] = pd.to_datetime(df["datetime_utc"], errors="coerce")

# datetime 파싱이 안 된 행 체크
bad_dates = df["datetime_utc"].isna().sum()
print("파싱 실패한 날짜 수:", bad_dates)

# 5) 정렬
df = df.sort_values(by="datetime_utc").reset_index(drop=True)

# 6) 저장
output_path = r"C:\Users\shinchaewon\Desktop\trump_tweets_2016_.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print("완성:", output_path)


df1 columns: ['datetime_utc', 'id', 'text', 'retweets', 'favorites']
df2 columns: ['datetime_utc', 'id', 'text', 'retweets', 'favorites', 'source']
합치기 전 전체 행수: 16924
중복 제거 후 행수: 16924
파싱 실패한 날짜 수: 364
완성: C:\Users\shinchaewon\Desktop\trump_tweets_2016_.csv
